In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from scml import pandasx as pdx

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
txn2id = {
    "clicks": 0,
    "carts": 1,
    "orders": 2,
}
rows = []
input_files = [Path("input/train.jsonl"), Path("input/test.jsonl")]
for fp in input_files:
    with open(str(fp)) as lines:
        for line in tqdm(lines, desc=str(fp)):
            jo = json.loads(line)
            session = int(jo["session"])
            prev = None
            for event in jo["events"]:
                curr = int(event["ts"])
                if prev is not None and curr<prev:
                    raise ValueError("event out-of-order")
                rows.append({
                    "session": session,
                    "aid": int(event["aid"]),
                    "ts": curr,
                    "type": txn2id[event["type"]]
                })
                prev = curr

input\train.jsonl: 12899779it [06:34, 32700.81it/s]
input\test.jsonl: 1671803it [00:16, 99062.85it/s] 


In [4]:
df = pd.DataFrame.from_records(rows)
cols = ["session", "aid"]
df[cols] = df[cols].astype(np.int32)
cols = ["type"]
df[cols] = df[cols].astype(np.int8)
#df.sort_values(["session", "ts"], inplace=True, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223644219 entries, 0 to 223644218
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   session  int32
 1   aid      int32
 2   ts       int64
 3   type     int8 
dtypes: int32(2), int64(1), int8(1)
memory usage: 3.5 GB


In [5]:
df.describe(percentiles=percentiles)

,session,aid,ts,type
count,2.236442e+08,2.236442e+08,2.236442e+08,2.236442e+08
mean,4.981796e+06,9.287208e+05,1.660580e+12,1.242825e-01
std,3.931718e+06,5.367068e+05,7.337569e+08,3.937267e-01
min,0.000000e+00,0.000000e+00,1.659305e+12,0.000000e+00
1%,4.246400e+04,1.763200e+04,1.659350e+12,0.000000e+00
5%,2.191720e+05,9.125600e+04,1.659449e+12,0.000000e+00
10%,4.949280e+05,1.822060e+05,1.659558e+12,0.000000e+00
20%,1.157152e+06,3.716300e+05,1.659823e+12,0.000000e+00
30%,1.949009e+06,5.585730e+05,1.660076e+12,0.000000e+00
40%,2.934384e+06,7.439770e+05,1.660336e+12,0.000000e+00


In [6]:
pdx.value_counts(df["type"])

,count,percent
0,201013586,0.898810
1,17466202,0.078098
2,5164431,0.023092


In [7]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/transactions.parquet", index=False)

Wall time: 17.4 s
